# OpenAI ChatGPT API와 LangChain 활용 가이드

## 개요

이 튜토리얼은 OpenAI 호환 Chat API 를 LangChain 과 함께 사용하는 방법을 설명한다. 실행 환경 구성, 기본 호출, 응답 구조, 스트리밍, 멀티모달, 프롬프트 설계를 단계별로 다룬다. 본 문서에서는 OpenRouter 를 통해 모델을 호출한다.

### 학습 목표

- **LangChain** 과 **LCEL(LangChain Expression Language)** 기본 개념 이해
- **ChatOpenAI** 클래스 초기화와 호출 흐름 이해
- **모델/파라미터 선택 기준** 이해
- **스트리밍, LogProb, 멀티모달** 사용법 습득
- **프롬프트 설계** 기본 원칙 정리

### 목차

1. 환경 설정과 LangSmith 연동
2. LangChain 과 LCEL 핵심 개념
3. ChatOpenAI 기본 사용법
4. 응답 구조 이해
5. 고급 기능: LogProb, 스트리밍, 멀티모달
6. 프롬프트 엔지니어링
---

## 환경 설정

In [41]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

True

In [42]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-Tutorial


---

## LangChain 과 LCEL 기본 개념

### LangChain 이란?

**LangChain** 은 Large Language Model(LLM) 기반 애플리케이션 개발을 위한 **통합 프레임워크** 이다.

#### LangChain 의 핵심 가치

- **표준화**: 다양한 LLM 을 동일한 인터페이스로 사용
- **구성요소화**: 프롬프트, 모델, 파서 등 모듈로 재사용
- **확장성**: 필요에 따라 컴포넌트 교체/추가 용이
- **추적성**: LangSmith 등과 연동하여 실행을 기록

### LCEL (LangChain Expression Language)

**LCEL** 은 LangChain 컴포넌트들을 **체인처럼 연결** 하는 표현식이다.

#### LCEL 의 특징

![](images/lcel.png)

```python
# LCEL 방식 - 간결하고 명확
chain = prompt | llm | output_parser
result = chain.invoke({"input": input_text})
```

#### LCEL 의 장점

- **가독성**: 처리 흐름을 한눈에 파악 가능
- **최적화 실행**: 내부적으로 병렬화 및 스트리밍 지원
- **견고성**: 예외 처리와 재시도 전략을 손쉽게 적용

---

## ChatOpenAI - OpenAI 호환 인터페이스

**ChatOpenAI** 는 OpenAI 호환 Chat API 를 LangChain 에서 사용할 수 있도록 제공하는 **통합 인터페이스** 이다. 본 튜토리얼에서는 OpenRouter 를 사용한다.

### 주요 설정 옵션

| 파라미터 | 목적 | 권장값(예시) |
| :-- | :-- | :-- |
| temperature | 출력 다양성 조절 | 0.1 |
| model_name | 사용할 모델 | openai/gpt-4.1 |
| api_key | 인증 키 | os.getenv('OPENROUTER_API_KEY') |
| base_url | API 엔드포인트 | os.getenv('OPENROUTER_BASE_URL') |
| max_tokens | 최대 출력 길이 | 필요 시 지정 |

#### temperature (창의성 조절)
```python
temperature=0.1  # 일관되고 정확한 답변 (사실 질문에 적합)
temperature=0.8  # 창의적이고 다양한 답변 (창작 활동에 적합)
```
- **범위**: 0.0 ~ 2.0
- **낮은 값 (0.0~0.3)**: 정확하고 일관된 답변
- **중간 값 (0.4~0.7)**: 일반적인 대화에 적합
- **높은 값 (0.8~2.0)**: 다양한 표현 생성

#### max_tokens (최대 출력 길이)
```python
max_tokens=2048
```
- **의미**: 생성할 토큰의 상한
- **참고**: 과도한 설정은 비용에 영향을 준다

#### model_name (사용할 모델)
- **GPT-4.1**: 고성능 모델, 복잡한 추론과 멀티모달 지원
- **GPT-4.1-mini**: 성능과 비용의 균형
- **GPT-4.1-nano**: 경량 작업에 적합

### OpenAI 모델 비교표

| 모델 계열       | 모델명 (API Name) | 입력       | 컨텍스트 윈도우  | 최대 출력 토큰 | 지식 마감일 (Cutoff) | 가격 (1M토큰당)                  |
| :---------- | :------------- | :------- | :-------- | :------- | :-------------- | :------------------------------ |
| **GPT-5**   | `gpt-5`        | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 9월 30일    | **입력:** $1.25<br>**출력:** $10.00 |
|             | `gpt-5-mini`   | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 5월 31일    | **입력:** $0.25<br>**출력:** $2.00  |
|             | `gpt-5-nano`   | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 5월 31일    | **입력:** $0.05<br>**출력:** $0.40  |
| **GPT-4.1** | `gpt-4.1`      | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $2.00<br>**출력:** $8.00  |
|             | `gpt-4.1-mini` | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $0.40<br>**출력:** $1.60  |
|             | `gpt-4.1-nano` | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $0.10<br>**출력:** $0.40  |
| **GPT-4o**  | `gpt-4o`       | 텍스트, 이미지 | 128,000   | 16,384   | 2023년 10월 1일    | **입력:** $2.50<br>**출력:** $10.00 |
|             | `gpt-4o-mini`  | 텍스트, 이미지 | 128,000   | 16,384   | 2023년 10월 1일    | **입력:** $0.15<br>**출력:** $0.60  |

![OpenAI Models Comparison](./images/gpt-models3-202508.png)

### 모델 선택 가이드

- **정확성 우선**: `gpt-4.1`
- **균형 선택**: `gpt-4.1-mini`
- **비용 절약**: `gpt-4.1-nano`

> **참고 링크**: [OpenAI 공식 모델 문서](https://platform.openai.com/docs/models)

### 기본 사용법

이제 ChatOpenAI 를 사용해 기본 호출을 수행한다.

In [43]:
from langchain_openai import ChatOpenAI
import os

# ChatOpenAI 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0) - 낮을수록 일관된 답변
    model_name="openai/gpt-4.1",  # 사용할 모델명 (OpenRouter)
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter API URL
)

# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LLM에 질의하고 결과 출력
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울특별시(서울)입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 16, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'id': 'chatcmpl-CkOr5vNIElH4aZq4QDPhg2PXxWRZI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--cd480a48-9775-423c-9d73-455a76bf343d-0' usage_metadata={'input_tokens': 16, 'output_tokens': 14, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### AIMessage 객체

ChatOpenAI 의 응답은 단순 텍스트가 아닌 **AIMessage 객체** 로 반환된다. 객체에는 본문과 메타데이터가 포함된다.

#### AIMessage 의 구성 요소

- **content**: 실제 AI 가 생성한 텍스트 답변
- **response_metadata**: 토큰 사용량, 모델 정보, 처리 시간 등 메타데이터
- **기타 정보**: 메시지 ID, 타입 등

In [44]:
# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LLM에 질의하고 응답 객체를 변수에 저장
response = llm.invoke(question)

In [45]:
# 전체 응답 객체 확인 (AIMessage 형태)
response

AIMessage(content='대한민국의 수도는 서울특별시(서울)입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 16, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'id': 'chatcmpl-CkOr6gyncvE5UBgbMZp4g5FIycOl1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--53b4980c-419a-4bcc-847a-47b2d0f1ff4f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 14, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [46]:
# 응답 내용만 텍스트로 추출
response.content

'대한민국의 수도는 서울특별시(서울)입니다.'

In [47]:
# 응답 메타데이터 확인 (토큰 사용량, 모델 정보 등)
response.response_metadata

{'token_usage': {'completion_tokens': 14,
  'prompt_tokens': 16,
  'total_tokens': 30,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4.1-2025-04-14',
 'system_fingerprint': 'fp_433e8c8649',
 'id': 'chatcmpl-CkOr6gyncvE5UBgbMZp4g5FIycOl1',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### LogProb - 토큰 선택 확률 정보

**LogProb(로그 확률)** 은 모델이 각 토큰을 선택할 때의 상대적 확률 정보를 제공한다. 답변의 신뢰도 판단 및 품질 관리에 활용할 수 있다.

#### 활용 예

- **답변 신뢰도 평가**: 낮은 확률 구간을 감지해 후속 확인 수행
- **품질 관리**: 불확실한 응답을 필터링하거나 재생성
- **모델 분석**: 토큰 수준의 선택 경향 파악
- **후처리**: 확률 기준으로 후보 답변 비교

#### 해석 방법 (경험칙)

- **높은 확률 (-0.1 ~ 0.0)**: 높은 확신
- **중간 확률 (-2.0 ~ -0.1)**: 보통 수준의 확신
- **낮은 확률 (-5.0 이하)**: 낮은 확신

In [61]:
# LogProb 기능이 활성화된 ChatOpenAI 객체 생성
import os

llm_with_logprob = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 출력 토큰 수
    model_name="openai/gpt-4.1",  # 사용할 모델명 (OpenRouter)
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter API URL
).bind(
    logprobs=True,
    top_logprobs=5,  # 상위 3개 토큰 확률 정보 포함
)  # 토큰별 확률 정보 활성화

In [62]:
# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LogProb가 활성화된 LLM으로 질의
response = llm_with_logprob.invoke(question)

In [63]:
# LogProb 정보가 포함된 메타데이터 확인
response_metadata = response.response_metadata

In [64]:
response_metadata["logprobs"]['content'][0]

{'token': '대한',
 'bytes': [235, 140, 128, 237, 149, 156],
 'logprob': -3.128163257315464e-07,
 'top_logprobs': [{'token': '대한',
   'bytes': [235, 140, 128, 237, 149, 156],
   'logprob': -3.128163257315464e-07},
  {'token': ' 대한민국',
   'bytes': [32, 235, 140, 128, 237, 149, 156, 235, 175, 188, 234, 181, 173],
   'logprob': -15.375},
  {'token': '네', 'bytes': [235, 132, 164], 'logprob': -18.375},
  {'token': '"', 'bytes': [34], 'logprob': -20.125},
  {'token': '대', 'bytes': [235, 140, 128], 'logprob': -20.375}]}

In [65]:
for i in response_metadata["logprobs"]['content']:
    print(i)

{'token': '대한', 'bytes': [235, 140, 128, 237, 149, 156], 'logprob': -3.128163257315464e-07, 'top_logprobs': [{'token': '대한', 'bytes': [235, 140, 128, 237, 149, 156], 'logprob': -3.128163257315464e-07}, {'token': ' 대한민국', 'bytes': [32, 235, 140, 128, 237, 149, 156, 235, 175, 188, 234, 181, 173], 'logprob': -15.375}, {'token': '네', 'bytes': [235, 132, 164], 'logprob': -18.375}, {'token': '"', 'bytes': [34], 'logprob': -20.125}, {'token': '대', 'bytes': [235, 140, 128], 'logprob': -20.375}]}
{'token': '민국', 'bytes': [235, 175, 188, 234, 181, 173], 'logprob': 0.0, 'top_logprobs': [{'token': '민국', 'bytes': [235, 175, 188, 234, 181, 173], 'logprob': 0.0}, {'token': '민', 'bytes': [235, 175, 188], 'logprob': -18.875}, {'token': '미', 'bytes': [235, 175, 184], 'logprob': -20.125}, {'token': ' 대한민국', 'bytes': [32, 235, 140, 128, 237, 149, 156, 235, 175, 188, 234, 181, 173], 'logprob': -21.875}, {'token': '한국', 'bytes': [237, 149, 156, 234, 181, 173], 'logprob': -22.625}]}
{'token': '의', 'bytes': [

In [66]:
import numpy as np

for token in response_metadata["logprobs"]["content"]:
    for i in token['top_logprobs']:
        token_str = i["token"].strip()
        logprob = float(i["logprob"])
        print(f"{token_str}\t\t {np.round(np.exp(logprob)*100, 2)}%")
    print("-----")
    # top_token_str = token["token"].strip()
    # top_logprob = float(token["logprob"])
    # print(f"TOP: {top_token_str}\t\t {np.round(np.exp(top_logprob)*100, 2)}%")

# # 출력 확률이 가장 높은 토큰만 출력
# for token in response_metadata["logprobs"]["content"]:
#     top_token_str = token["token"].strip()
#     top_logprob = float(token["logprob"])
#     print(f"TOP: {top_token_str}\t\t {np.round(np.exp(top_logprob)*100, 2)}%")


대한		 100.0%
대한민국		 0.0%
네		 0.0%
"		 0.0%
대		 0.0%
-----
민국		 100.0%
민		 0.0%
미		 0.0%
대한민국		 0.0%
한국		 0.0%
-----
의		 99.97%
(		 0.03%
(K		 0.0%
の		 0.0%
,		 0.0%
-----
수도		 100.0%
수		 0.0%
**		 0.0%
		 0.0%
공식		 0.0%
-----
는		 100.0%
は		 0.0%
은		 0.0%
는		 0.0%
(		 0.0%
-----
**		 49.92%
서울		 49.92%
"		 0.16%
'		 0.01%
"**		 0.0%
-----
특		 90.36%
입니다		 9.52%
(		 0.11%
특별		 0.01%
Special		 0.0%
-----
별		 100.0%
別		 0.0%
별		 0.0%
别		 0.0%
변		 0.0%
-----
시		 100.0%
시장		 0.0%
시에		 0.0%
市		 0.0%
십		 0.0%
-----
(		 85.17%
입니다		 14.8%
,		 0.03%
입니다		 0.0%
(S		 0.0%
-----
서울		 98.82%
Se		 1.1%
줄		 0.08%
일		 0.0%
약		 0.0%
-----
)		 99.97%
,		 0.03%
시		 0.0%
또는		 0.0%
;		 0.0%
-----
입니다		 100.0%
입니다		 0.0%
です		 0.0%
​		 0.0%
﻿		 0.0%
-----
.		 100.0%
.		 0.0%
.​		 0.0%
.		 0.0%
<|end|>		 0.0%
-----


In [68]:
import numpy as np

for token in response_metadata["logprobs"]["content"]:
    token_str = token["token"].strip()
    logprob = float(token["logprob"])
    print(f"{token_str}\t\t {np.round(np.exp(logprob)*100, 2)}%")

대한		 100.0%
민국		 100.0%
의		 99.97%
수도		 100.0%
는		 100.0%
서울		 49.92%
특		 90.36%
별		 100.0%
시		 100.0%
(		 85.17%
서울		 98.82%
)		 99.97%
입니다		 100.0%
.		 100.0%


### 스트리밍 출력

스트리밍은 모델이 생성하는 토큰을 순차적으로 전송해, 응답을 실시간으로 확인할 수 있도록 한다. 긴 답변도 대기 시간을 줄여 빠르게 피드백을 받을 수 있다.

#### 장점

- **지연 감소**: 부분 결과를 즉시 확인
- **과정 노출**: 생성 진행 상황 파악
- **UX 향상**: 대화형 애플리케이션에 적합

#### 활용 예

- **긴 문서 작성 미리보기**
- **실시간 챗봇 응답**
- **데모/발표 환경**

In [ ]:
# 스트리밍 방식으로 LLM에 질의
# 실시간으로 토큰이 생성되는 과정을 확인할 수 있음
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

In [ ]:
# 스트리밍 응답을 실시간으로 출력
# 각 토큰이 생성될 때마다 즉시 화면에 표시됨
for token in answer:
    print(token.content, end="", flush=True)

In [ ]:
from langchain_teddynote.messages import stream_response

# 스트리밍 방식으로 LLM에 질의
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

# langchain_teddynote의 stream_response 함수로 깔끔하게 출력
stream_response(answer)

## 멀티모달 AI - 이미지를 읽는 인공지능

**멀티모달(Multimodal)** 은 여러 종류의 데이터를 동시에 처리하는 기술이다. **텍스트, 이미지, 오디오, 비디오** 등 복수 입력을 이해한다.

### 처리 가능한 데이터 타입

- **텍스트**: 문서, 이메일, 웹페이지 등
- **이미지**: 사진, 차트, 표, 스크린샷 등
- **오디오**: 음성, 음악, 효과음 등
- **비디오**: 동영상, 애니메이션 등

### GPT-4.1 의 비전(Vision) 기능

**GPT-4.1** 은 강력한 **이미지 인식 능력** 을 갖춘 멀티모달 모델이다.

#### 이미지 분석 작업 예

- **차트/그래프 해석**: 데이터 시각화 분석
- **문서 OCR**: 이미지 속 텍스트 추출 및 해석
- **장면 설명**: 사진 속 상황과 객체 인식
- **표/양식 처리**: 복잡한 테이블 데이터 이해
- **시각 자료 분석**: 그림, 디자인 요소 해석

#### 비즈니스 활용 예

- **재무제표 분석**: 복잡한 회계 자료 자동 해석
- **의료 영상 검토**: X-ray, MRI 이미지 보조 분석
- **건축 도면 검토**: 설계도 및 시공 현황 파악
- **제품 관리**: 상품 사진을 통한 품질 검사

In [ ]:
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 기본 ChatOpenAI 객체 생성
import os

llm = ChatOpenAI(
    temperature=0.1,  # 창의성
    model_name="openai/gpt-4.1",  # 이미지 인식이 가능한 모델 (OpenRouter)
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter API URL
)

# 멀티모달(이미지 + 텍스트 처리) 객체 생성
multimodal_llm = MultiModal(llm)

In [ ]:
# 웹상의 이미지 URL 정의
IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# 웹 이미지를 직접 분석하여 스트리밍 응답 생성
answer = multimodal_llm.stream(IMAGE_URL)

# 실시간으로 이미지 분석 결과 출력
stream_response(answer)

In [ ]:
# 로컬 저장된 이미지 파일 경로 정의
IMAGE_PATH_FROM_FILE = "01-LCEL/images/sample-image.png"

# 로컬 이미지 파일을 분석하여 스트리밍 응답 생성
answer = multimodal_llm.stream(IMAGE_PATH_FROM_FILE)

# 실시간으로 이미지 분석 결과 출력
stream_response(answer)

In [ ]:
# 시스템 프롬프트: AI의 역할과 행동 방식을 정의
system_prompt = """You are a professional financial AI assistant specialized in analyzing financial statements and tables. 
Your mission is to interpret given tabular financial data and provide insightful, interesting findings in a friendly and helpful manner. 
Focus on key metrics, trends, and notable patterns that would be valuable for business analysis.

[IMPORTANT]
- 한글로 답변해 주세요.
"""

# 사용자 프롬프트: 구체적인 작업 지시사항
user_prompt = """Please analyze the financial statement provided in the image. 
Identify and summarize the most interesting and important findings, including key financial metrics, trends, and insights that would be valuable for business decision-making."""

# 커스텀 프롬프트가 적용된 멀티모달 객체 생성
multimodal_llm_with_prompt = MultiModal(
    llm,
    system_prompt=system_prompt,  # 시스템 역할 정의
    user_prompt=user_prompt,  # 사용자 요청 정의
)

In [ ]:
# 분석할 재무제표 이미지 URL
IMAGE_PATH_FROM_FILE = "https://storage.googleapis.com/static.fastcampus.co.kr/prod/uploads/202212/080345-661/kwon-01.png"

# 커스텀 프롬프트가 적용된 멀티모달 LLM으로 재무제표 분석
answer = multimodal_llm_with_prompt.stream(IMAGE_PATH_FROM_FILE)

# 재무제표 분석 결과를 실시간으로 출력
stream_response(answer)